In [8]:
import json
import numpy as np
import pandas as pd
import unidecode
from pandas.io.json import json_normalize


## Creating the news dataframe
The first step is import the news dataset and creating a Pandas Dataframe

In [9]:
news_data_file = open('../datasets/raw-data/folha_de_sao_paulo_news.json', 'r')
news_json = json.load(news_data_file)

news_df = json_normalize(news_json['news'])

news_df.head(5)

,authors,categories,date_published,link,locations,news_body,sub_title,title
0,"[gustavo uribe, ricardo della coletta]","[poder, governo_bolsonaro, lava_jato, legislat...",2019-08-29 16:01:00,https://www1.folha.uol.com.br/poder/2019/08/em...,[brasília],"na tentativa de negar que tenha havido um , o ...",governo escalou 4 governadores e 12 ministros ...,"em esforço para negar desgaste, bolsonaro cham..."
1,"[daniel carvalho, thiago resende]","[mercado, mercado_financeiro, previdência, tec...",2019-08-28 12:10:00,https://www1.folha.uol.com.br/mercado/2019/08/...,[brasília],a pauta econômica tornou-se motivo para uma no...,imbróglio começa pela disputa por protagonismo...,câmara e senado travam nova guerra por pauta e...
2,"[felipe bächtold, wálter nunes, reynaldo turol...","[poder, governo_bolsonaro, lava_jato, legislat...",2019-08-28 08:09:00,https://www1.folha.uol.com.br/poder/2019/08/se...,[são_paulo],a sentença que condenou o ex-presidente lula n...,"nesta terça-feira, supremo anulou pela primeir...",sentença em caso do sítio usado por lula é sem...
3,[ranier bragon],"[poder, governo_bolsonaro, lava_jato, legislat...",2019-08-27 19:52:00,https://www1.folha.uol.com.br/poder/2019/08/so...,[brasília],após desgaste político e em meio à situação cr...,congresso irá definir no final do ano valor da...,"sob pressão, relator desiste da proposta de en..."
4,"[daniel carvalho, thiago resende]","[poder, governo_bolsonaro, lava_jato, legislat...",2019-08-28 12:30:00,https://www1.folha.uol.com.br/poder/2019/08/co...,[brasília],"diante da , parlamentares e articuladores do g...",presidentes de câmara e senado buscam apazigua...,congresso tenta blindar reformas e projetos de...


# Selecting only the columns that we are going to feed into the Neural Network

In [10]:
news_df['text'] = news_df['title'] + ". " + news_df['sub_title'] + ". " + news_df['news_body']
news_df = news_df[['date_published', 'text']]

news_df.head(5)

,date_published,text
0,2019-08-29 16:01:00,"em esforço para negar desgaste, bolsonaro cham..."
1,2019-08-28 12:10:00,câmara e senado travam nova guerra por pauta e...
2,2019-08-28 08:09:00,sentença em caso do sítio usado por lula é sem...
3,2019-08-27 19:52:00,"sob pressão, relator desiste da proposta de en..."
4,2019-08-28 12:30:00,congresso tenta blindar reformas e projetos de...


# Saving the clean dataset into a file

In [11]:
news_df.to_csv('../datasets/pre-processed/news.csv', index= False, encoding='utf-8')